In [11]:
import tensorflow as tf
from session_recommendation.hgru4rec.user_par_mini_batch import UserParallelMiniBatchDataset
from tensorflow.contrib.cudnn_rnn import CudnnGRU
from tensorflow.keras.layers import GRU, Dense, Dropout
from session_recommendation.hgru4rec.hgru4rec import HGRU4Rec
import numpy as np

In [2]:
params = dict()
params['num_units_session'] = 25
params['num_units_user'] = 50
params['num_products'] = 542346
params['num_users'] = 307526
params['embedding_size'] = 25
params['user_rnn_layers'] = 2
params['user_rnn_units'] = 50
params['session_rnn_layers'] = 2
params['session_rnn_units'] = 25
params['num_negative_samples'] = 10
params['user_dropout'] = 0.1
params['session_dropout'] = 0.1
params['init_dropout'] = 0.1

In [3]:
# get datapoint iterator
dataset = UserParallelMiniBatchDataset(10, 'gs://ma-muy/03_datasets/midi_dataset/05_train/')
datapoints = dataset.feature_and_label_generator()

/home/muy/.local/share/virtualenvs/code-JIyQgSPM/lib/python3.6/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [6]:
batch = next(datapoints)
batch

,EmbeddingId,LastSessionEvent,ProductId,SessionChanged,SessionId,Timestamp,UserChanged,UserEmbeddingId,UserId,LabelEmbeddingId
0,15007,0,7022501,0,2294769417,1537532024,0,10920,3052507,15008
1,8007,0,2584445,0,1935612609,1527064869,0,10769,1396107,12622
2,3054,1,2470752,0,2242223061,1536164879,0,12048,1078807,3054
3,7636,1,6664094,0,2284073729,1537270494,0,12057,1894007,3700
4,10941,1,2481055,0,1939928352,1527194690,0,11104,2562807,7675
5,944,0,7513881,0,2781821874,1543692508,0,11869,966907,1547
6,483,1,8766371,0,2401965647,1540021323,0,12039,642707,11575
7,11273,0,5738828,0,2211018839,1535302767,0,11814,270707,16879
8,35849,1,2584235,0,2267312850,1536836822,0,12067,2754207,350
9,32903,0,209625,0,2197060372,1534939529,0,11162,700707,25318


In [7]:
model_instance = HGRU4Rec(params)

In [8]:
preprocessed = model_instance._preprocess(batch)

In [13]:
preprocessed = preprocessed.astype(dtype={'SessionChanged': np.bool})
preprocessed

,index,EmbeddingId,LastSessionEvent,ProductId,SessionChanged,SessionId,Timestamp,UserChanged,UserEmbeddingId,UserId,LabelEmbeddingId,UserEmbedding,SessionEmbedding
0,0,15007,0,7022501,False,2294769417,1537532024,0,10920,3052507,15008,"[2.1002574456377383, 1.0810521835487734, 0.710...","[-1.5092821519750488, 0.1699316939430801, -0.5..."
1,1,8007,0,2584445,False,1935612609,1527064869,0,10769,1396107,12622,"[-0.0022552431773172005, -0.8967108582235661, ...","[0.8880969790369566, 0.3209600599962996, 0.074..."
2,5,944,0,7513881,False,2781821874,1543692508,0,11869,966907,1547,"[0.20383387113914378, 3.3909118703028525, 1.03...","[-1.0081373799956257, -0.07003160018105213, 0...."
3,7,11273,0,5738828,False,2211018839,1535302767,0,11814,270707,16879,"[-1.8499015596523118, -0.7614199526667907, -0....","[1.5651396133348614, 0.14334714912039134, 0.85..."
4,9,32903,0,209625,False,2197060372,1534939529,0,11162,700707,25318,"[0.21223907347824056, 1.1307897374705091, -1.3...","[1.46710182317162, -0.14958971568627108, -0.60..."


In [19]:
preprocessed['LabelEmbeddingId'].values.shape

(5,)

In [20]:
np.array([[1,2],[1,2],[1,2]]).shape

(3, 2)

In [3]:
tf.enable_eager_execution()

In [8]:
labels

{'ProductId': <tf.Tensor: id=372, shape=(10,), dtype=int64, numpy=
 array([3230492, 6515651, 6353140, 6184828, 3514849, 6165296, 6050625,
        6883686,  203623, 8191687])>,
 'EmbeddingId': <tf.Tensor: id=371, shape=(10,), dtype=int64, numpy=
 array([ 8643,   478,  6460,   192, 14870,   119,  4132,  2198, 18719,
          978])>}

In [6]:
features['SessionId']

<tf.Tensor: id=367, shape=(10,), dtype=int64, numpy=
array([2297336104, 2315256845, 2261966035, 2011006476, 2002847394,
       2476312908, 2344520139, 2029829736, 2045943822, 2069504154])>

In [43]:
with tf.variable_scope('test', tf.AUTO_REUSE):
    a = tf.get_variable(
        'num_ended_sessions',
        shape=(),
        initializer=tf.zeros_initializer(),
        trainable=False,
        dtype=tf.int64)
    
    print(a)
    
    a = tf.assign(
        a,
        tf.add(tf.reduce_sum(features['SessionChanged']),a))
    
    print(a)
    
    a = tf.get_variable(
        'num_ended_sessions',
        shape=(),
        initializer=tf.zeros_initializer(),
        trainable=False,
        dtype=tf.int64)
    
    print(a)

<tf.Variable 'test/num_ended_sessions:0' shape=() dtype=int64, numpy=0>
<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=2>
<tf.Variable 'test/num_ended_sessions:0' shape=() dtype=int64, numpy=0>


In [15]:
batch_size = features['UserId'].shape[0]

num_ended_sessions = tf.get_variable(
    'num_ended_sessions',
    shape=(),
    initializer=tf.zeros_initializer(),
    trainable=False,
    dtype=tf.int64)

num_ended_users = tf.get_variable(
    'num_ended_users',
    shape=(),
    initializer=tf.zeros_initializer(),
    trainable=False,
    dtype=tf.int64)

# Hidden states of session_rnn
session_hidden_states = tf.get_variable(
    'session_hidden_states',
    shape=(batch_size, params['session_rnn_units']),
    initializer=tf.zeros_initializer(),
    trainable=False)

# User Embedding, updated by user_rnn
user_embeddings = tf.get_variable(
    'user_embeddings',
    shape=(params['num_users'], params['user_rnn_units']),
    initializer=tf.zeros_initializer(),
    trainable=False)

# Softmax weights to map RNN output to product space
softmax_weights = tf.get_variable(
    'softmax_weights',
    shape=(params['num_products'], params['session_rnn_units']))

# Biases for above
softmax_biases = tf.get_variable(
    'softmax_biases',
    shape=(params['num_products'],))

In [16]:
user_rnn = GRU(
    # params['user_rnn_layers'],
    params['user_rnn_units'],
    return_state=True,
    implementation=2,
    dropout=params['user_dropout'],
    kernel_initializer=tf.contrib.layers.xavier_initializer(),
    recurrent_initializer=tf.contrib.layers.xavier_initializer(),
    name='user_rnn')

session_rnn = GRU(
    # params['session_rnn_layers'],
    params['session_rnn_units'],
    return_state=True,
    implementation=2,
    dropout=params['session_dropout'],
    kernel_initializer=tf.contrib.layers.xavier_initializer(),
    recurrent_initializer=tf.contrib.layers.xavier_initializer(),
    name='session_rnn')

# Layer to predict new session initialization
user2session_layer = Dense(
    params['session_rnn_units'],
    input_shape=(params['user_rnn_units'],),
    activation='tanh',
    name='user2session_layer')

# Dropout layer for session initialization
user2session_dropout = Dropout(params['init_dropout'])

In [21]:
# Update stats
num_ended_sessions = tf.add(
    tf.reduce_sum(features['SessionChanged']),
    num_ended_sessions)

num_ended_users = tf.add(
    tf.reduce_sum(features['UserChanged']),
    num_ended_users)

print(num_ended_sessions)
print(num_ended_users)

tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)


In [30]:
tf.get_variable(
    'num_ended_sessions',
    shape=(),
    initializer=tf.zeros_initializer(),
    trainable=False,
    dtype=tf.int64)

<tf.Variable 'num_ended_sessions:0' shape=() dtype=int64, numpy=0>

In [19]:
# Get session hidden states to update
indices_to_update = tf.squeeze(
    tf.where(
        tf.cast(features['SessionChanged'], tf.bool),
        name='get_indices_to_update'
    ),
    axis=1
)

session_states_to_update = tf.gather(
    session_hidden_states,
    indices_to_update,
    name='get_session_states_to_update'
)

print(indices_to_update)
print(session_states_to_update.shape)

tf.Tensor([5 9], shape=(2,), dtype=int64)
(2, 25)


In [22]:
# Get user embeddings to update
user_embedding_ids_to_update = tf.gather(
    features['UserEmbeddingId'],
    indices_to_update
)

user_embeddings_to_update = tf.nn.embedding_lookup(
    user_embeddings,
    user_embedding_ids_to_update
)

print(user_embedding_ids_to_update)
print(user_embeddings_to_update.shape)

tf.Tensor([ 482 1389], shape=(2,), dtype=int64)
(2, 50)


In [23]:
# Compute new user representation for all users in current batch
new_session_hidden_states_seed, new_user_embeddings = user_rnn.apply(
    tf.expand_dims(session_states_to_update, 1),
    initial_state=user_embeddings_to_update)

# Predict new session initialization for next session
new_session_hidden_states = user2session_layer.apply(
    new_session_hidden_states_seed)

new_session_hidden_states = user2session_dropout.apply(
    new_session_hidden_states)

print(new_session_hidden_states.shape)
print(new_user_embeddings.shape)
print(new_session_hidden_states)

(2, 25)
(2, 50)
tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]], shape=(2, 25), dtype=float32)


In [52]:
scattered_new_states = tf.scatter_nd(
        tf.cast(tf.expand_dims(indices_to_update, axis=1), tf.int32),
        new_session_hidden_states,
        tf.shape(session_hidden_states)
    )
print(scattered_new_states.shape)

(10, 25)


In [26]:
# Reset Session Hidden States to 0 when a user has ended
session_hidden_states = tf.where(
    tf.cast(features['UserChanged'], tf.bool),
    tf.zeros(tf.shape(session_hidden_states)),
    session_hidden_states,
    name='reset_session_hidden_states')

In [48]:
# Compute mask for missing users
existing_user_indices = tf.squeeze(tf.where(tf.not_equal(features['UserId'], -1)))

print(existing_user_indices)

# Compute mask for ending sessions
contiuing_session_indices = tf.squeeze(tf.where(
        tf.logical_not(
            tf.cast(features['LastSessionEvent'], tf.bool))))

print(contiuing_session_indices)

# Compute relevant indices
relevant_indices = tf.squeeze(
    tf.sparse.to_dense(
        tf.sets.intersection(
            existing_user_indices[None, :],
            contiuing_session_indices[None, :]
        )
    ),
    axis=0)

print(relevant_indices)

tf.Tensor([0 1 2 3 4 5 6 7 8 9], shape=(10,), dtype=int64)
tf.Tensor([0 2 3 4 5 7 8 9], shape=(8,), dtype=int64)
tf.Tensor([0 2 3 4 5 7 8 9], shape=(8,), dtype=int64)


In [12]:
# Compute new user representation for all users in current batch
new_session_hidden_states_seed, new_user_hidden_states = user_rnn.apply(
    tf.expand_dims(session_hidden_states, 1),
    initial_state=user_hidden_states)

# Predict new session initialization for next session
new_session_hidden_states = user2session_layer.apply(
    new_session_hidden_states_seed)

new_session_hidden_states = user2session_dropout.apply(
    new_session_hidden_states
)

In [13]:
# Select new session initialization for new sessions
session_hidden_states = tf.where(
    ended_sessions_mask,
    new_session_hidden_states,
    session_hidden_states,
    name='initialize_new_sessions')

# Update user hidden states where the session ended
user_hidden_states = tf.where(
    ended_sessions_mask,
    new_user_hidden_states,
    user_hidden_states,
    name='update_user_representation'
)

In [26]:
# Compute new mask for ended sessions
ended_sessions_mask = tf.cast(
    tf.where(
        tf.equal(features['ProductId'], -1),
        tf.ones(tf.shape(ended_sessions_mask)),
        tf.zeros(tf.shape(ended_sessions_mask)),
        name='compute_ended_sessions'),
    tf.bool)

# Compute new mask for ending sessions
ending_sessions_mask = tf.cast(
    tf.where(
        tf.equal(labels['ProductId'], -1),
        tf.ones(tf.shape(ending_sessions_mask)),
        tf.zeros(tf.shape(ending_sessions_mask)),
        name='compute_ending_sessions'),
    tf.bool)

# Compute new mask for ended users
ended_users_mask = tf.cast(
    tf.where(
        tf.equal(features['UserId'], -1),
        tf.ones(tf.shape(ended_users_mask)),
        tf.zeros(tf.shape(ended_users_mask)),
        name='compute_ended_users'),
    tf.bool)

In [27]:
# Relevant sessions have not ended and do not end in the next step
relevant_sessions_mask = tf.logical_not(
    tf.logical_or(
        ended_sessions_mask,
        ending_sessions_mask))

# Get one-hot encoding of products
relevant_one_hots = tf.map_fn(
    lambda x: tf.cond(
        x[1],
        lambda: tf.one_hot(x[0], params['num_products']),
        lambda: tf.zeros(params['num_products'])
    ),
    [
        features['EmbeddingId'],
        relevant_sessions_mask
    ],
    dtype=tf.float32,
    name='get_relevant_one_hots')

# Get session hidden states for relevant sessions
relevant_hidden_states = tf.where(
    relevant_sessions_mask,
    session_hidden_states,
    tf.zeros(tf.shape(session_hidden_states)),
    name='get_relevant_session_hidden_states'
)

In [28]:
relevant_one_hots.shape

TensorShape([Dimension(10), Dimension(2600000)])

In [29]:
# Apply Session RNN -> get new hidden states and predictions
predictions, new_session_hidden_states = session_rnn.apply(
    tf.expand_dims(relevant_one_hots, 1),
    initial_state=relevant_hidden_states)

In [21]:
# Filter out irrelevant predictions
predictions = tf.boolean_mask(
    predictions,
    relevant_sessions_mask,
    name='filter_irrelevant_predictions')

# Update session hidden states for relevant sessions
session_hidden_states = tf.where(
    relevant_sessions_mask,
    new_session_hidden_states,
    session_hidden_states,
    name='update_relevant_session_hidden_states')

# Extract relevant labels
relevant_labels = tf.boolean_mask(
    labels['EmbeddingId'],
    relevant_sessions_mask,
    name='filter_irrelevant_labels')

# Compute logits for product predictions
logits = tf.matmul(
    predictions,
    softmax_weights,
    transpose_b=True) + softmax_biases

# Apply softmax activation
softmax_predictions = tf.nn.softmax(logits)

In [22]:
tf.shape(softmax_predictions)

<tf.Tensor: id=1184, shape=(2,), dtype=int32, numpy=array([     10, 2600000], dtype=int32)>

In [43]:
# Compute Hitrate

in_top_k = tf.nn.in_top_k(softmax_predictions, relevant_labels, 5)
hitrate = tf.divide(
    tf.reduce_sum(tf.cast(in_top_k, tf.int64)),
    tf.shape(labels)[0])

In [47]:
# Compute Loss Function

negative_samples_weights = tf.nn.embedding_lookup(softmax_weights, relevant_labels)
negative_samples_biases = tf.nn.embedding_lookup(softmax_biases, relevant_labels)

logits = tf.matmul(predicted_embeddings, negative_samples_weights, transpose_b=True) + negative_samples_biases
yhat = tf.nn.softmax(logits) # for each of the examples in the batch we select the remainder of the minibatch as negative examples

# TOP 1 Loss function
yhatT = tf.transpose(yhat)
term1 = tf.reduce_mean(tf.nn.sigmoid(-tf.diag_part(yhat)+yhatT)+tf.nn.sigmoid(yhatT**2), axis=0)
term2 = tf.nn.sigmoid(tf.diag_part(yhat)**2) / batch_size.value
loss = tf.reduce_mean(term1 - term2)

In [48]:
predi

<tf.Tensor: id=2867, shape=(), dtype=float32, numpy=0.9640579>

In [55]:
# Optimize
optimizer = tf.train.AdamOptimizer()
train_op = optimizer.minimize(loss, global_step=tf.train.get_or_create_global_step())

RuntimeError: `loss` passed to Optimizer.compute_gradients should be a function when eager execution is enabled.

In [16]:
session_hidden_states

<tf.Tensor: id=639, shape=(10, 25), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
      